When using a <font color='blue'>regression to fit a model to our data, the assumptions of regression analysis </font>must be satisfied in order to <font color='blue'>ensure good parameter estimates and accurate fit statistics. 

<font color='blue'>We would like parameters to be:

- <font color='blue'>unbiased (expected value over different samples is the true value)
- <font color='blue'>consistent (converging to the true value with many samples), and
- <font color='blue'>efficient (minimized variance)

Rather than focusing on your model construction, <font color='blue'>it is possible to gain a huge amount of information from your residuals (errors).

Your model may be incredibly complex and impossible to analyze, but <font color='blue'>as long as you have predictions and observed values, you can compute residuals. Once you have your residuals you can perform many statistical tests.

<font color='blue'>If your residuals do not follow a given distribution (usually normal, but depends on your model), then you know that something is wrong and you should be concerned with the accuracy of your predictions.

<font color='blue'>If the error term is not normally distributed, then our tests of statistical significance will be off. 

<font color='blue'>Fortunately, the central limit theorem tells us that, for large enough data samples, the coefficient distributions will be close to normal even if the errors are not. Therefore our analysis will still be valid for large data datasets.

<font color='blue'>A good test for normality is the Jarque-Bera test. It has a python implementation at `statsmodels.stats.stattools.jarque_bera` , we will use it frequently in this notebook.

In [ ]:
import statsmodels.api as sm
from statsmodels import regression, stats
import statsmodels

In [ ]:
residuals = np.random.poisson(size = 100)
_, pvalue, _, _ = statsmodels.stats.stattools.jarque_bera(residuals)
print pvalue

<font color='blue'>Heteroskedasticity means that the variance of the error terms is not constant across observations. 

Intuitively, this means that the <font color='blue'>observations are not uniformly distributed along the regression line. 

It often occurs in <font color='blue'>cross-sectional data where the differences in the samples we are measuring lead to differences in the variance.

In [ ]:
# Get results of linear regression
slr1 = regression.linear_model.OLS(y1, sm.add_constant(xs)).fit()
# Construct the fit line
fit1 = slr1.params[0] + slr1.params[1]*xs


<font color='blue'>You can test for heteroskedasticity using a few tests, we'll use the Breush Pagan test from the statsmodels library. 

We'll also test for normality, which in this case also picks up the weirdness in the second case. HOWEVER, it is possible to have normally distributed residuals which are also heteroskedastic, so both tests must be performed to be sure.

In [ ]:
xs_with_constant = sm.add_constant(xs)
_, pvalue1, _, _ = stats.diagnostic.het_breushpagan(residuals1, xs_with_constant)
_, pvalue2, _, _ = stats.diagnostic.het_breushpagan(residuals2, xs_with_constant)
print "p-value for residuals1 being heteroskedastic", pvalue1
print "p-value for residuals2 being heteroskedastic", pvalue2

<font color='blue'>The problematic situation, known as conditional heteroskedasticity, is when the error variance is correlated with the independent variables as it is above. 
    
This makes the F-test for regression significance and t-tests for the significances of individual coefficients unreliable. Most often this results in overestimation of the significance of the fit.

<font color='blue'>The Breusch-Pagan test and the White test can be used to detect conditional heteroskedasticity. 

<font color='blue'>If we suspect that this effect is present, we can alter our model to try and correct for it. 

<font color='blue'>One method is generalized least squares, which requires a manual alteration of the original equation. 

<font color='blue'>Another is computing robust standard errors, which corrects the fit statistics to account for the heteroskedasticity. 

In [ ]:
print slr2.summary()
print slr2.get_robustcov_results().summary()

<font color='blue'>A common and serious problem is when errors are correlated across observations (known serial correlation or autocorrelation). </font>This can occur, for instance, <font color='blue'>when some of the data points are related, or when we use time-series data with periodic fluctuations. 

<font color='blue'>If one of the independent variables depends on previous values of the dependent variable - such as when it is equal to the value of the dependent variable in the previous period - or if incorrect model specification leads to autocorrelation, then the coefficient estimates will be inconsistent and therefore invalid. 

<font color='blue'>Otherwise, the parameter estimates will be valid, but the fit statistics will be off.</font> For instance, if the correlation is positive, we will have inflated F- and t-statistics, leading us to overestimate the significance of the model.

<font color='blue'>If the errors are homoskedastic, we can test for autocorrelation using the Durbin-Watson test, which is conveniently reported in the regression summary in `statsmodels`.

In [ ]:
# Load pricing data for an asset
start = '2014-01-01'
end = '2015-01-01'
y = get_pricing('DAL', fields='price', start_date=start, end_date=end)
x = np.arange(len(y))
# Regress pricing data against time
model = regression.linear_model.OLS(y, sm.add_constant(x)).fit()
# Construct the fit line
prediction = model.params[0] + model.params[1]*x
# Plot pricing data and regression line
plt.plot(x,y)
plt.plot(x, prediction, color='r')
plt.legend(['DAL Price', 'Regression Line'])
plt.xlabel('Time')
plt.ylabel('Price')
# Print summary of regression results
model.summary()

We can <font color='blue'>test for autocorrelation</font> in both our prices and residuals. We'll use the built-in method to do this, which is based on the <font color='blue'>Ljun-Box test

<font color='blue'>This test computes the probability that the n-th lagged datapoint is predictive of the current. If no max lag is given, then the function computes a max lag and returns the p-values for all lags up to that one.

In [ ]:
_, prices_qstats, prices_qstat_pvalues = statsmodels.tsa.stattools.acf(y, qstat=True)
_, prices_qstats, prices_qstat_pvalues = statsmodels.tsa.stattools.acf(y-prediction, qstat=True)
print 'Prices autocorrelation p-values', prices_qstat_pvalues
print 'Residuals autocorrelation p-values', prices_qstat_pvalues

<font color='blue'>Newey-West is a method of computing variance which accounts for autocorrelation. A naive variance computation will actually produce inaccurate standard errors with the presence of autocorrelation.

<font color='blue'>We can attempt to change the regression equation to eliminate serial correlation. A simpler fix is adjusting the standard errors using an appropriate method and using the adjusted values to check for significance.

Below we use the Newey-West method from `statsmodels` to compute adjusted standard errors for the coefficients. They are higher than those originally reported by the regression, which is what we expected for positively correlated errors.

In [ ]:
from math import sqrt
# Find the covariance matrix of the coefficients
cov_mat = stats.sandwich_covariance.cov_hac(model)
# Print the standard errors of each coefficient from the original model and from the adjustment
print 'Old standard errors:', model.bse[0], model.bse[1]
print 'Adjusted standard errors:', sqrt(cov_mat[0,0]), sqrt(cov_mat[1,1])

<font color='blue'>When using multiple independent variables, it is important to check for multicollinearity; that is, an approximate linear relation between the independent variables

<font color='blue'>As with truly unnecessary variables, this will usually not hurt the accuracy of the model, but will cloud our analysis. In particular, the estimated coefficients will have large standard errors.

High correlation between independent variables is indicative of multicollinearity. <font color='blue'>However, it is not enough, since we would want to detect correlation between one of the variables and a linear combination of the other variables. 

<font color='blue'>If we have high R-squared but low t-statistics on the coefficients (the fit is good but the coefficients are not estimated precisely) we may suspect multicollinearity. 

<font color='blue'>To resolve the problem, we can drop one of the independent variables involved in the linear relation.

For instance, using two stock indices as our independent variables is likely to lead to multicollinearity. Below we can see that removing one of them improves the t-statistics without hurting R-squared.

<font color='blue'>Another important thing to determine here is which variable may be the casual one. </font>If we hypothesize that the market influences both MDY and HPQ, then the market is the variable that we should use in our predictive model.

In [ ]:
# Load pricing data for asset and two market indices
start = '2014-01-01'
end = '2015-01-01'
b1 = get_pricing('SPY', fields='price', start_date=start, end_date=end)
b2 = get_pricing('MDY', fields='price', start_date=start, end_date=end)
a = get_pricing('HPQ', fields='price', start_date=start, end_date=end)
# Run multiple linear regression
mlr = regression.linear_model.OLS(a, sm.add_constant(np.column_stack((b1,b2)))).fit()
# Construct fit curve using dependent variables and estimated coefficients
mlr_prediction = mlr.params[0] + mlr.params[1]*b1 + mlr.params[2]*b2
# Print regression statistics 
print 'R-squared:', mlr.rsquared_adj
print 't-statistics of coefficients:\n', mlr.tvalues
# Plot asset and model
a.plot()
mlr_prediction.plot()
plt.legend(['Asset', 'Model']);
plt.ylabel('Price')

In [ ]:
slr = regression.linear_model.OLS(a, sm.add_constant(b1)).fit()
slr_prediction = slr.params[0] + slr.params[1]*b1
# Print fit statistics
print 'R-squared:', slr.rsquared_adj
print 't-statistics of coefficients:\n', slr.tvalues
# Plot asset and model
a.plot()
slr_prediction.plot()
plt.ylabel('Price')
plt.legend(['Asset', 'Model']);

In [ ]:
from scipy.stats import pearsonr
# Construct Anscombe's arrays
x1 = [10, 8, 13, 9, 11, 14, 6, 4, 12, 7, 5]
y1 = [8.04, 6.95, 7.58, 8.81, 8.33, 9.96, 7.24, 4.26, 10.84, 4.82, 5.68]
x2 = [10, 8, 13, 9, 11, 14, 6, 4, 12, 7, 5]
y2 = [9.14, 8.14, 8.74, 8.77, 9.26, 8.10, 6.13, 3.10, 9.13, 7.26, 4.74]
x3 = [10, 8, 13, 9, 11, 14, 6, 4, 12, 7, 5]
y3 = [7.46, 6.77, 12.74, 7.11, 7.81, 8.84, 6.08, 5.39, 8.15, 6.42, 5.73]
x4 = [8, 8, 8, 8, 8, 8, 8, 19, 8, 8, 8]
y4 = [6.58, 5.76, 7.71, 8.84, 8.47, 7.04, 5.25, 12.50, 5.56, 7.91, 6.89]

# Perform linear regressions on the datasets
slr1 = regression.linear_model.OLS(y1, sm.add_constant(x1)).fit()
slr2 = regression.linear_model.OLS(y2, sm.add_constant(x2)).fit()
slr3 = regression.linear_model.OLS(y3, sm.add_constant(x3)).fit()
slr4 = regression.linear_model.OLS(y4, sm.add_constant(x4)).fit()

# Print regression coefficients, Pearson r, and R-squared for the 4 datasets
print 'Cofficients:', slr1.params, slr2.params, slr3.params, slr4.params
print 'Pearson r:', pearsonr(x1, y1)[0], pearsonr(x2, y2)[0], pearsonr(x3, y3)[0], pearsonr(x4, y4)[0]
print 'R-squared:', slr1.rsquared, slr2.rsquared, slr3.rsquared, slr4.rsquared

# Plot the 4 datasets with their regression lines
f, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2,2)
xs = np.arange(20)
ax1.plot(slr1.params[0] + slr1.params[1]*xs, 'r')
ax1.scatter(x1, y1)
ax1.set_xlabel('x1')
ax1.set_ylabel('y1')
ax2.plot(slr2.params[0] + slr2.params[1]*xs, 'r')
ax2.scatter(x2, y2)
ax2.set_xlabel('x2')
ax2.set_ylabel('y2')
ax3.plot(slr3.params[0] + slr3.params[1]*xs, 'r')
ax3.scatter(x3, y3)
ax3.set_xlabel('x3')
ax3.set_ylabel('y3')
ax4.plot(slr4.params[0] + slr4.params[1]*xs, 'r')
ax4.scatter(x4,y4)
ax4.set_xlabel('x4')
ax4.set_ylabel('y4');